In [2]:
from datasets import load_dataset
from glob import glob

file_locs = sorted(glob("../data/raw/*"))
print(file_locs)
files_ds = [load_dataset("json", data_files=file_loc) for file_loc in file_locs]
ds = files_ds[1]
ds["test"] = files_ds[0]["train"]
ds["valid"] = files_ds[2]["train"]
ds

['../data/raw/streaminqa_eval.jsonl', '../data/raw/streaminqa_train.jsonl', '../data/raw/streaminqa_valid.jsonl']


Using custom data configuration default-787a8af077014e5f
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-787a8af077014e5f/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-72c4eb2145caad09
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-72c4eb2145caad09/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-e34becee86a825d2
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e34becee86a825d2/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 99402
    })
    test: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 36378
    })
    valid: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_to

In [3]:
# Import t5 for conditional generation
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [4]:

# dataset encode the dataset
ds = ds.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512), batched=True , num_proc=60)


#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#4:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#5:   0%|          | 0/2 [00:00<?, ?ba/s]

KeyError: 'text'

In [5]:
ds["train"][0]

{'qa_id': 'train-000000',
 'question': 'When was George W. Bush\'s memoir, "Decision Points," published?',
 'answers': ['November 9, 2010'],
 'answers_additional': [],
 'question_ts': 1490338800,
 'evidence_ts': 1488700800,
 'evidence_id': '20170305000000000000\x00\x011c98344aaf81280d4f1dae80f8ca31a6318a40040d7f2ce803ff455959800723\x00\x01',
 'recent_or_past': 'recent',
 'written_or_generated': 'generated',
 'toxicity_identity_attack': 0.014725318178534508,
 'toxicity_insult': 0.02712947130203247,
 'toxicity_profanity': 0.01666371151804924,
 'toxicity_severe_toxicity': 0.01330364029854536,
 'toxicity_sexually_explicit': 0.026551254093647003,
 'toxicity_threat': 0.02643008343875408}

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

Was ist die Capitale France?


In [11]:

# inference
input_ids = tokenizer(
    "Question: What is the Capital of France ? Answer: ", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# studies have shown that owning a dog is good for you.

Answer:


In [19]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'EleutherAI/gpt-neo-1.3B')

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [20]:
generator("What is the Capital of USA ?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'What is the Capital of USA ? The capital city of USA has got a name and a special location in the world. The Capital of USA is USA Capital City.\n\nUSA Capital City, USA Capital City refers to the principal city, which is'}]